<a href="https://colab.research.google.com/github/owenlutz4/STATS_415_Project/blob/main/CNN-LSTM_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os

print("Loading data...")

try:
    # Load training data
    X_train = pd.read_csv("UCI_data/train/X_train.txt", sep='\s+', header=None)
    y_train = pd.read_csv("UCI_data/train/y_train.txt", header=None)
    subject_train = pd.read_csv("UCI_data/train/subject_train.txt", header=None)

    # Load test data
    X_test = pd.read_csv("UCI_data/test/X_test.txt", sep='\s+', header=None)
    y_test = pd.read_csv("UCI_data/test/y_test.txt", header=None)
    subject_test = pd.read_csv("UCI_data/test/subject_test.txt", header=None)

    print("Initial data shapes:")
    print(f"X_train shape: {X_train.shape}")
    print(f"y_train shape: {y_train.shape}")
    print(f"X_test shape: {X_test.shape}")
    print(f"y_test shape: {y_test.shape}")
except Exception as e:
    print("Failed to load data:", str(e))
    raise

# Normalize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# One-hot encode labels
encoder = OneHotEncoder(sparse_output=False)
y_train_encoded = encoder.fit_transform(y_train)
y_test_encoded = encoder.transform(y_test)

# Get labels for the stratification
y_train_classes = np.argmax(y_train_encoded, axis=1)

# Calculate dimensions
n_timesteps = 11
n_features = 51
n_classes = y_train_encoded.shape[1]

# Reshape data for CNN-LSTM
X_train_reshaped = X_train_scaled.reshape(-1, n_timesteps, n_features)
X_test_reshaped = X_test_scaled.reshape(-1, n_timesteps, n_features)

def create_model(conv_filters, lstm_units, dropout_rate):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(n_timesteps, n_features)),
        tf.keras.layers.Conv1D(
            filters=conv_filters,
            kernel_size=5,
            activation='relu',
            kernel_regularizer=tf.keras.regularizers.l2(0.0005)
        ),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.MaxPooling1D(pool_size=2),
        tf.keras.layers.LSTM(
            lstm_units,
            return_sequences=False,
            kernel_regularizer=tf.keras.regularizers.l2(0.0005),
            recurrent_regularizer=tf.keras.regularizers.l2(0.0005)
        ),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(
            32,
            activation='relu',
            kernel_regularizer=tf.keras.regularizers.l2(0.0005)
        ),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(
          n_classes,
          activation='softmax',
          kernel_regularizer=tf.keras.regularizers.l2(0.0005)
        )
    ])
    return model

# Define parameter grid
param_grid = {
    'conv_filters': [32, 64],
    'lstm_units': [32, 64],
    'dropout_rate': [0.25, 0.35],
    'learning_rate': [0.0001, 0.0005],
    'batch_size': [64, 128]
}

def grid_search_cv():
    best_accuracy = 0
    best_params = {}
    results = []

    total_combinations = (len(param_grid['conv_filters']) *
                        len(param_grid['lstm_units']) *
                        len(param_grid['dropout_rate']) *
                        len(param_grid['learning_rate']) *
                        len(param_grid['batch_size']))

    print(f"Starting grid search with {total_combinations} combinations...")
    combination_count = 0

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    try:
        for conv_filters in param_grid['conv_filters']:
            for lstm_units in param_grid['lstm_units']:
                for dropout_rate in param_grid['dropout_rate']:
                    for learning_rate in param_grid['learning_rate']:
                        for batch_size in param_grid['batch_size']:
                            combination_count += 1
                            print(f"\nTrying combination {combination_count}/{total_combinations}")
                            print(f"Parameters: conv_filters={conv_filters}, lstm_units={lstm_units}, "
                                  f"dropout_rate={dropout_rate}, lr={learning_rate}, batch_size={batch_size}")

                            cv_scores = []

                            for fold, (train_idx, val_idx) in enumerate(skf.split(X_train_reshaped, y_train_classes)):
                                print(f"Fold {fold+1}/5")

                                X_fold_train = X_train_reshaped[train_idx]
                                y_fold_train = y_train_encoded[train_idx]
                                X_fold_val = X_train_reshaped[val_idx]
                                y_fold_val = y_train_encoded[val_idx]

                                try:
                                    model = create_model(conv_filters, lstm_units, dropout_rate)
                                    model.compile(
                                        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                                        loss='categorical_crossentropy',
                                        metrics=['accuracy']
                                    )

                                    model.fit(
                                        X_fold_train,
                                        y_fold_train,
                                        epochs=50,
                                        batch_size=batch_size,
                                        validation_data=(X_fold_val, y_fold_val),
                                        callbacks=[
                                            tf.keras.callbacks.EarlyStopping(
                                                monitor='val_loss',
                                                patience=3,
                                                restore_best_weights=True,
                                                min_delta=0.005
                                            )
                                        ],
                                        verbose=0
                                    )

                                    _, fold_accuracy = model.evaluate(X_fold_val, y_fold_val, verbose=0)
                                    cv_scores.append(fold_accuracy)

                                except tf.errors.ResourceExhaustedError:
                                    print(f"Skipping fold {fold+1} due to memory error")
                                    continue
                                finally:
                                    tf.keras.backend.clear_session()

                            if cv_scores:
                                mean_cv_accuracy = np.mean(cv_scores)
                                std_cv_accuracy = np.std(cv_scores)
                                print(f"\nMean CV accuracy: {mean_cv_accuracy:.4f} (+/- {std_cv_accuracy:.4f})")

                                results.append({
                                    'conv_filters': conv_filters,
                                    'lstm_units': lstm_units,
                                    'dropout_rate': dropout_rate,
                                    'learning_rate': learning_rate,
                                    'batch_size': batch_size,
                                    'mean_cv_accuracy': mean_cv_accuracy,
                                    'std_cv_accuracy': std_cv_accuracy
                                })

                                if mean_cv_accuracy > best_accuracy:
                                    best_accuracy = mean_cv_accuracy
                                    best_params = {
                                        'conv_filters': conv_filters,
                                        'lstm_units': lstm_units,
                                        'dropout_rate': dropout_rate,
                                        'learning_rate': learning_rate,
                                        'batch_size': batch_size
                                    }
                                    print(f"\nNew best accuracy: {best_accuracy:.4f}")

    except KeyboardInterrupt:
        print("\nGrid search interrupted by user")
        if best_params:
            return best_params, best_accuracy, results
        raise

    return best_params, best_accuracy, results

print("Starting grid search with 5-fold cross validation...")
best_params, best_accuracy, all_results = grid_search_cv()

print("\nGrid search complete!")
print(f"Best parameters found:")
print(f"- Conv filters: {best_params['conv_filters']}")
print(f"- LSTM units: {best_params['lstm_units']}")
print(f"- Dropout rate: {best_params['dropout_rate']}")
print(f"- Learning rate: {best_params['learning_rate']}")
print(f"- Batch size: {best_params['batch_size']}")
print(f"Best CV accuracy: {best_accuracy:.4f}")

# Train final model
print("\nTraining final model with best parameters...")
final_model = create_model(
    best_params['conv_filters'],
    best_params['lstm_units'],
    best_params['dropout_rate']
)

final_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=best_params['learning_rate']),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
base_dir = 'model_outputs'
output_dir = f'{base_dir}_{timestamp}'
os.makedirs(output_dir, exist_ok=True)

final_history = final_model.fit(
    X_train_reshaped,
    y_train_encoded,
    epochs=50,
    batch_size=best_params['batch_size'],
    validation_split=0.2,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True,
            min_delta=0.005
        ),
        tf.keras.callbacks.ModelCheckpoint(
            'best_model.weights.h5',
            monitor='val_loss',
            save_best_only=True,
            mode='min',
            save_weights_only=True
        )
    ],
    verbose=1
)

print("\nEvaluating final model...")
test_loss, test_accuracy = final_model.evaluate(X_test_reshaped, y_test_encoded)
print(f"Final Test Accuracy: {test_accuracy:.4f}")

y_pred = final_model.predict(X_test_reshaped)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test_encoded, axis=1)

activity_labels = [
    'WALKING',
    'WALKING_UPSTAIRS',
    'WALKING_DOWNSTAIRS',
    'SITTING',
    'STANDING',
    'LAYING'
]

print("\nClassification Report:")
print(classification_report(y_test_classes, y_pred_classes, target_names=activity_labels))

def plot_confusion_matrix(y_true, y_pred, classes, filename):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(12, 10))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=classes, yticklabels=classes)
    plt.title('Confusion Matrix of Human Activities')
    plt.ylabel('True Activity')
    plt.xlabel('Predicted Activity')
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig(filename, bbox_inches='tight', dpi=300)
    plt.close()

plot_confusion_matrix(
    y_test_classes,
    y_pred_classes,
    activity_labels,
    os.path.join(output_dir, 'confusion_matrix.png')
)

# Accuracy History Plot

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(final_history.history['accuracy'], label='Training')
plt.plot(final_history.history['val_accuracy'], label='Validation')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Loss History Plot

plt.subplot(1, 2, 2)
plt.plot(final_history.history['loss'], label='Training')
plt.plot(final_history.history['val_loss'], label='Validation')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'training_history.png'))
plt.close()

try:
    model_path = os.path.join(output_dir, 'final_model.h5')
    final_model.save(model_path)
    print(f"Model saved to: {model_path}")
except Exception as e:
    print(f"Error saving model: {str(e)}")
    try:
        backup_path = os.path.join(output_dir, 'final_model_backup')
        final_model.save(backup_path, save_format='tf')
        print(f"Model saved to backup location: {backup_path}")
    except Exception as e2:
        print(f"Failed to save backup model: {str(e2)}")

results_df = pd.DataFrame(all_results)
results_df.to_csv(os.path.join(output_dir, 'grid_search_results.csv'), index=False)

print(f"\nOutputs saved to: {output_dir}")

def cleanup():
   tf.keras.backend.clear_session()
   plt.close('all')
   if os.path.exists('best_model.weights.h5'):
       os.remove('best_model.weights.h5')

import atexit
atexit.register(cleanup)

if __name__ == "__main__":
   try:
       print("Starting training process...")
   except Exception as e:
       print(f"Error in main process: {str(e)}")
   finally:
       cleanup()

In [ ]:
import time
import psutil
import numpy as np
import os
from datetime import datetime

def measure_inference_performance(model, X_test, n_runs=100):
    """
    Measure inference time and memory usage of the model.

    Args:
        model: Trained TensorFlow model
        X_test: Test data
        n_runs: Number of inference runs to average over

    Returns:
        dict: Performance metrics
    """
    # Warm up the model
    for _ in range(5):
        model.predict(X_test[:1])

    # Measure inference time
    latencies = []
    memory_usage = []

    for _ in range(n_runs):
        # Single sample inference (real-time scenario)
        start_time = time.time()
        model.predict(X_test[:1])
        latencies.append((time.time() - start_time) * 1000)  # Convert to milliseconds

        # Memory usage
        process = psutil.Process()
        memory_usage.append(process.memory_info().rss / 1024 / 1024)  # Convert to MB

    metrics = {
        'avg_inference_time_ms': np.mean(latencies),
        'std_inference_time_ms': np.std(latencies),
        'max_inference_time_ms': np.max(latencies),
        'p95_inference_time_ms': np.percentile(latencies, 95),
        'avg_memory_usage_mb': np.mean(memory_usage),
        'peak_memory_usage_mb': np.max(memory_usage)
    }

    return metrics

# Add to model evaluation section:
print("\nMeasuring real-time performance metrics...")
performance_metrics = measure_inference_performance(final_model, X_test_reshaped)

print("\nReal-time Performance Metrics:")
print(f"Average Inference Time: {performance_metrics['avg_inference_time_ms']:.2f} ms")
print(f"95th Percentile Inference Time: {performance_metrics['p95_inference_time_ms']:.2f} ms")
print(f"Maximum Inference Time: {performance_metrics['max_inference_time_ms']:.2f} ms")
print(f"Average Memory Usage: {performance_metrics['avg_memory_usage_mb']:.2f} MB")
print(f"Peak Memory Usage: {performance_metrics['peak_memory_usage_mb']:.2f} MB")

# Create output directory if it doesn't exist
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
base_dir = 'model_outputs'
output_dir = f'{base_dir}_{timestamp}'
os.makedirs(output_dir, exist_ok=True)

# Save metrics to inference results file
with open(os.path.join(output_dir, 'inference_results.txt'), 'w') as f:
    f.write('Real-time Performance Metrics:\n')
    for metric, value in performance_metrics.items():
        f.write(f"{metric}: {value:.2f}\n")